In [1]:
import json
import torch
import torch.nn.functional as F
import clip
from pprint import pprint

import sys

sys.path.append("..")
from models import Linear

clip_model_name = "ViT-B/32"
clip_model, transform = clip.load(name=clip_model_name, device="cuda")
clip_model = clip_model.float()

# Waterbird

In [2]:
linear_model_path = "../pytorch_cache/iclrsubmission/models/waterbird_linear_model.pt"
data_path = "../../data/Waterbird/processed_attribute_dataset/attributes.jsonl"
state_dict = torch.load(linear_model_path)
n_class = state_dict["fc.weight"].shape[0]
model = Linear(clip_model.visual.output_dim, n_class).cuda()
model.load_state_dict(state_dict)

image_data = [
    json.loads(line)
    for line in open(
        "../../data/Waterbird/processed_attribute_dataset/attributes.jsonl"
    )
]
attributes = {
    "place": set([x["attributes"]["place"] for x in image_data]),
    "species": set([x["attributes"]["species"] for x in image_data]),
}

place_list = list(attributes["place"])
species_list = list(attributes["species"])
attribute_shapley = {}
for place in place_list:
    prompts = [f"a photo of a {species}." for species in species_list] + [
        f"a photo of a {species} in the {place}." for species in species_list
    ]
    with torch.no_grad():
        inputs = clip.tokenize(prompts).cuda()
        embeddings = F.normalize(clip_model.encode_text(inputs))
        probs = torch.softmax(model(embeddings), dim=1).cpu()

    shapley = (probs[len(species_list) :, 1] - probs[: len(species_list), 1]).mean()
    attribute_shapley[place] = shapley

print(
    "If we care about waterbird (label = 1), the most influential attributes are (reported Shapley value):"
)
pprint(sorted(attribute_shapley.items(), key=lambda x: x[1], reverse=True)[:10])

If we care about waterbird (label = 1), the most influential attributes are (reported Shapley value):
[('ocean', tensor(0.3062)),
 ('lake natural', tensor(0.0713)),
 ('forest broadleaf', tensor(-0.1540)),
 ('bamboo forest', tensor(-0.1931))]


# FairFace

In [3]:
linear_model_path = "../pytorch_cache/iclrsubmission/models/fairface_linear_model.pt"
data_path = "../../data/FairFace/processed_attribute_dataset/attributes.jsonl"
state_dict = torch.load(linear_model_path)
n_class = state_dict["fc.weight"].shape[0]
model = Linear(clip_model.visual.output_dim, n_class).cuda()
model.load_state_dict(state_dict)

image_data = [
    json.loads(line)
    for line in open("../../data/FairFace/processed_attribute_dataset/attributes.jsonl")
]
age_description = {
    "0-2": "infant",
    "3-9": "little",
    "10-19": "teenage",
    "20-29": "young",
    "30-39": "adult",
    "40-49": "middle-aged",
    "50-59": "senior",
    "60-69": "elderly",
    "more than 70": "very old",
}
attributes = {
    "race": set(
        [x["attributes"]["race"].lower().replace("_", " ") for x in image_data]
    ),
    "age": set([age_description[x["attributes"]["age"]] for x in image_data]),
}

race_list = list(attributes["race"])
age_list = list(attributes["age"])
attribute_shapley = {}
for race in race_list:
    prompts = (
        [f"a face of a {age} man." for age in age_list]
        + [f"a face of a {age} woman." for age in age_list]
        + [f"a face of a {race} {age} man." for age in age_list]
        + [f"a face of a {race} {age} woman." for age in age_list]
    )
    with torch.no_grad():
        inputs = clip.tokenize(prompts).cuda()
        embeddings = F.normalize(clip_model.encode_text(inputs))
        probs = torch.softmax(model(embeddings), dim=1).cpu()

    shapley = (probs[len(age_list) * 2 :, 1] - probs[: 2 * len(age_list), 1]).mean()
    attribute_shapley[race] = shapley
print(
    "If we care about gender (label = 1 female), the most influential attributes are (reported Shapley value):"
)
pprint(sorted(attribute_shapley.items(), key=lambda x: x[1], reverse=True)[:10])

If we care about gender (label = 1 female), the most influential attributes are (reported Shapley value):
[('east asian', tensor(0.0266)),
 ('southeast asian', tensor(0.0119)),
 ('indian', tensor(0.0116)),
 ('latino hispanic', tensor(0.0046)),
 ('white', tensor(0.0040)),
 ('middle eastern', tensor(3.8439e-05)),
 ('black', tensor(-0.0327))]


# dSprites

In [4]:
linear_model_path = (
    "../pytorch_cache/iclrsubmission/models/dsprites_linear_model_2class.pt"
)
data_path = "../../data/TriangleSquare/processed_attribute_dataset/attributes.jsonl"
state_dict = torch.load(linear_model_path)
n_class = state_dict["fc.weight"].shape[0]
model = Linear(clip_model.visual.output_dim, n_class).cuda()
model.load_state_dict(state_dict)

attributes = {
    "scale": {"small", "medium", "large"},
    "color": {"blue", "cyan", "green", "orange", "pink", "red"},
}

scale_list = list(attributes["scale"])
color_list = list(attributes["color"])
attribute_shapley = {}
for color in color_list:
    prompts = (
        [f"a {scale} square." for scale in scale_list]
        + [f"a {scale} triangle." for scale in scale_list]
        + [f"a {scale} {color} square." for scale in scale_list]
        + [f"a {scale} {color} triangle." for scale in scale_list]
    )
    with torch.no_grad():
        inputs = clip.tokenize(prompts).cuda()
        embeddings = F.normalize(clip_model.encode_text(inputs))
        probs = torch.softmax(model(embeddings), dim=1).cpu()

    shapley = (probs[len(scale_list) * 2 :, 1] - probs[: 2 * len(scale_list), 1]).mean()
    attribute_shapley[color] = shapley
print(
    "If we care about triangle (label = 1), the most influential attributes are (reported Shapley value):"
)
pprint(sorted(attribute_shapley.items(), key=lambda x: x[1], reverse=True)[:10])

If we care about triangle (label = 1), the most influential attributes are (reported Shapley value):
[('orange', tensor(0.3736)),
 ('red', tensor(-0.0470)),
 ('blue', tensor(-0.0784)),
 ('cyan', tensor(-0.0979)),
 ('pink', tensor(-0.1181)),
 ('green', tensor(-0.3321))]
